# Analyzing neuropil signals one experiment at a time 
* 10/9/23 setup
* 10/16/23 migrating to `sleep` repo and stripping of sima stuff
* 10/18/23 kinda works for a sinle mouse
* refactoring with dfof calculation
* 10/30/23 adding dfof calculation

In [ ]:
import sys, os
import matplotlib.pyplot as plt
from os.path import join
import pandas as pd
import scipy.signal as signal
from scipy.interpolate import interp1d

# sys.path.append("/home/gergely/code/sleep/src")

from src.classes.suite2p_class import Suite2p

from src.classes.imaging_data_class import ImagingData
from src.classes.eeg_class import eegData
from src.classes.behavior_class import behaviorData
import src.classes.dfof


def freq_calc(data: pd.Series, fs: int = 10):
    """data: npil_eeg data"""
    frequencies, psd = signal.welch(data, fs=fs, detrend="linear")
    return frequencies, psd

### initializations of necessary classes

In [ ]:
mice = ["5HT2afl05b_1", "5HT2afl05b_2", "sert52b_1", "sert52b_5", "sert54a_5"]

In [ ]:
mouse = ImagingData(mice[0])
mouse.mouse_id
s2p_folders = mouse.find_s2p_folders()
for folder in enumerate(s2p_folders):
    print(folder)

In [ ]:
s2p_folder = s2p_folders[9]
sima_folder = os.path.dirname(s2p_folder)
sima_folder

In [ ]:
file_name = join(sima_folder, "eeg", "velo_eeg.csv")
eeg_velo = eegData.load_processed_velocity_eeg(file_name=file_name)

eeg_velo["mobile_immobile"] = behaviorData.define_mobility(eeg_velo["filtered velo"])
eeg_velo

In [ ]:
df = eegData.brain_state_filter(
    velo_eeg_df=eeg_velo,
    states=["awake_mobile", "awake_immobile", "NREM", "REM", "other"],
)
df

### Loading raw fluorescence data then calculating dfof

In [ ]:
s2p_calc_dfof = dfof.Suite2pDFOF(window=1800)
jia_calc_dfof = dfof.JiaDFOF(slow_trend_window=1800)
s2p_data = Suite2p(s2p_folder=s2p_folder)
s2p_data
cells = s2p_data.get_cells()
npil = s2p_data.get_npil()

In [ ]:
cell_dfof = s2p_calc_dfof.calculate(signal=cells, npil=npil)
npil_dfof = jia_calc_dfof.calculate(signal=npil)

### Looking at only the NREM vs awake immobile periods

#### Cell signal NREM vs. Awake immobile

In [ ]:
mean_cellF = cell_dfof.mean(axis=0)
fig, ax = plt.subplots(nrows=3, figsize=(20, 5), sharey=True)
# concatentate the True values:
ax[0].plot(mean_cellF.values, color="k", label="all cells")
ax[0].legend(loc="upper right", labelcolor="k")
ax[1].plot(mean_cellF[df["awake_immobile"]].values, label="awake")
ax[1].legend(loc="upper right", labelcolor="k")
ax[2].plot(mean_cellF[df["NREM"]].values, color="orange")
ax[2].legend(["NREM"], loc="upper right", labelcolor="k")
plt.show()
# fig.savefig(join(sima_folder, 'NREM Awake_immob Mean cell traces.png'),
#             bbox_inches='tight', dpi=300)

### Npil signal NREM vs. Awake immobile

#### NREM vs Awake immobile

In [ ]:
mean_npilF = npil_dfof.mean(axis=0)
fig, ax = plt.subplots(nrows=3, figsize=(20, 5), sharey=True)
ax[0].plot(mean_npilF.values, color="k")
ax[0].legend(["all npil"], loc="upper right", labelcolor="k")
ax[1].plot(mean_npilF[df["awake_immobile"]].values, label="awake")
ax[1].legend(labelcolor="k")
ax[1].legend(loc="upper right", labelcolor="k")
ax[2].plot(mean_npilF[df["NREM"]].values, color="orange", label="NREM")
ax[2].legend(["NREM"], loc="upper right", labelcolor="k")
plt.show()
# fig.savefig(
#     join(sima_folder, "NREM Awake_immob Mean npil traces.png"),
#     bbox_inches="tight",
#     dpi=300,
# )

### Calculations

In [ ]:
input_data = {
    "nrem_cell": pd.Series(mean_cellF[df["NREM"]].values),
    "nrem_npil": pd.Series(mean_npilF[df["NREM"]].values),
    "awake_npil": pd.Series(mean_npilF[df["awake_immobile"]].values),
    "awake_cell": pd.Series(mean_cellF[df["awake_immobile"]].values),
}

In [ ]:
results = {}

for key, value in input_data.items():
    freq, psd = freq_calc(value)
    results[key] = {"freq": freq, "psd": psd}

results.keys()

In [ ]:
# Create a figure with 2 subplots in 1 row and 2 columns
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5), sharey=True)
fig.suptitle("Power Spectral Density: Awake vs NREM")

# Plot the first semilogy plot in the first subplot
axs[0].semilogy(
    results["awake_cell"]["freq"], results["awake_cell"]["psd"], label="awake"
)
axs[0].semilogy(results["nrem_cell"]["freq"], results["nrem_cell"]["psd"], label="nrem")
axs[0].legend(loc="lower left")

# Plot the second semilogy plot in the first subplot
axs[1].semilogy(
    results["awake_npil"]["freq"], results["awake_npil"]["psd"], label="awake"
)
axs[1].semilogy(results["nrem_npil"]["freq"], results["nrem_npil"]["psd"], label="nrem")
axs[1].legend(loc="lower left")

# Set the title and axis labels for the first subplot
axs[0].set_title("Cell signal")
axs[0].set_xlabel("Frequency (Hz)")
axs[0].set_ylabel("Power/Frequency (dB/Hz)")

# Set the title and axis labels for the second subplot
axs[1].set_title("Npil signal")
axs[1].set_xlabel("Frequency (Hz)")

# Show the plot
plt.show()
# fig.savefig(join(sima_folder, 'PSD plot cell vs npil.png'),
#             bbox_inches='tight', dpi=300)

In [ ]:
# Create a figure with 2 subplots in 1 row and 2 columns
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5), sharey=True)
fig.suptitle("Power Spectral Density: Cell vs Npil")

# Plot the first semilogy plot in the first subplot
axs[0].semilogy(
    results["awake_cell"]["freq"], results["awake_cell"]["psd"], label="cell"
)
axs[0].semilogy(
    results["awake_npil"]["freq"], results["awake_npil"]["psd"], label="npil"
)
axs[0].legend()

# Plot the second semilogy plot in the first subplot
axs[1].semilogy(results["nrem_cell"]["freq"], results["nrem_cell"]["psd"], label="cell")
axs[1].semilogy(results["nrem_npil"]["freq"], results["nrem_npil"]["psd"], label="npil")
axs[1].legend()

# Set the title and axis labels for the first subplot
axs[0].set_title("Awake signal")
axs[0].set_xlabel("Frequency (Hz)")
axs[0].set_ylabel("Power/Frequency (dB/Hz)")

# Set the title and axis labels for the second subplot
axs[1].set_title("NREM signal")
axs[1].set_xlabel("Frequency (Hz)")

# Show the plot
plt.show()
# fig.savefig(join(sima_folder, 'PSD plot awake vs NREM.png'),
#             bbox_inches='tight', dpi=300)

### Old-style calcs

### NREM vs non-NREM

In [ ]:
cells = s2p_mouse.true_signal(s2p_folder=s2p_folder)
# cells = s2p_mouse.true_cells(s2p_folder)
mean_cellF = pd.DataFrame(cells.mean(axis=0))
mean_cellF
fig, ax = plt.subplots(nrows=2, figsize=(20, 5))
ax[0].plot(
    signal.detrend(mean_cellF[~df["NREM"]], axis=0, type="linear"),
    color="orange",
    label="awake",
)
ax[0].legend(loc="upper right", labelcolor="k")
ax[1].plot(signal.detrend(mean_cellF[df["NREM"]], axis=0, type="linear"))
ax[1].legend(["NREM"], loc="upper right", labelcolor="k")
plt.show()
# fig.savefig(join(sima_folder, 'NREM Awake Mean cell traces.png'),
#             bbox_inches='tight', dpi=300)

### NREM vs. non-NREM

In [ ]:
cells = s2p_mouse.true_signal(s2p_folder=s2p_folder, signal_source="Fneu")
mean_npilF = pd.DataFrame(cells.mean(axis=0))
mean_npilF
fig, ax = plt.subplots(nrows=2, figsize=(20, 5))
ax[0].plot(
    signal.detrend(mean_npilF[~df["NREM"]], axis=0, type="linear"),
    color="orange",
    label="awake",
)
ax[0].legend(loc="upper right", labelcolor="k")
ax[0].legend(labelcolor="k")
ax[1].plot(signal.detrend(mean_npilF[df["NREM"]], axis=0, type="linear"), label="NREM")
ax[1].legend(["NREM"], loc="upper right", labelcolor="k")
plt.show()
fig.savefig(
    join(sima_folder, "NREM Awake Mean npil traces.png"), bbox_inches="tight", dpi=300
)